## Ejercicio

### Calcular la cantidad de existencias en el inventario de sets del tema Star Wars.

## Bases: sets, inventory & themes

### Desarrollo:

In [1]:
import pandas

In [29]:
sets = pandas.read_csv('MATERIALES-CAP2/02-merging-dataframes/lego-database/sets.csv.gz')
inventory_sets = pandas.read_csv('MATERIALES-CAP2/02-merging-dataframes/lego-database/inventory_sets.csv.gz')
themes = pandas.read_csv('MATERIALES-CAP2/02-merging-dataframes/lego-database/themes.csv.gz')

In [5]:
sets.head()

,set_num,name,year,theme_id,num_parts
0,001-1,Gears,1965,1,43
1,0011-2,Town Mini-Figures,1978,84,12
2,0011-3,Castle 2 for 1 Bonus Offer,1987,199,0
3,0012-1,Space Mini-Figures,1979,143,12
4,0013-1,Space Mini-Figures,1979,143,12


In [6]:
inventory_sets.head()

,inventory_id,set_num,quantity
0,35,75911-1,1
1,35,75912-1,1
2,39,75048-1,1
3,39,75053-1,1
4,50,4515-1,1


In [18]:
themes.head()

,id,name,parent_id
0,1,Technic,NaN
1,2,Arctic Technic,1.0
2,3,Competition,1.0
3,4,Expert Builder,1.0
4,5,Model,1.0


In [30]:
#Para que haya relación entre sets y themes, tengo que cambiar la columna de themes id por themes_id
themes = themes.rename(columns = {'id' : 'theme_id'})
themes.head()

,theme_id,name,parent_id
0,1,Technic,NaN
1,2,Arctic Technic,1.0
2,3,Competition,1.0
3,4,Expert Builder,1.0
4,5,Model,1.0


In [32]:
# Primero realizamos la relación entre set y theme, luego eliminamos la columna porent_id:
set_themes = pandas.merge(sets, themes[['theme_id', 'name']], suffixes=('_set', '_theme'), on = 'theme_id')

set_themes.head()

,set_num,name_set,year,theme_id,num_parts,name_theme
0,001-1,Gears,1965,1,43,Technic
1,002-1,4.5V Samsonite Gears Motor Set,1965,1,3,Technic
2,1030-1,TECHNIC I: Simple Machines Set,1985,1,191,Technic
3,1038-1,ERBIE the Robo-Car,1985,1,120,Technic
4,1039-1,Manual Control Set 1,1986,1,39,Technic


#### Ahora calcularemos la cantidad de elementos que hay para inventories:

In [37]:
quantity = inventory_sets.get(['set_num', 'quantity']).groupby('set_num', as_index = False). sum()

quantity.head()

,set_num,quantity
0,001-1,1
1,10000-1,1
2,10001-1,1
3,10002-1,1
4,10013-1,1


#### Una vez que ya tenemos las cantidad y los sets con los temas correspondientes, pasaremos a relacionarlo para saber la intersección con el merge final:

In [43]:
themes_quantity = pandas.merge(set_themes, quantity, on = 'set_num', validate = '1:1')

themes_quantity.head(10)

,set_num,name_set,year,theme_id,num_parts,name_theme,quantity
0,001-1,Gears,1965,1,43,Technic,1
1,9610-1,Gear Set,1993,1,76,Technic,12
2,9612-1,Lever Set,1993,1,65,Technic,12
3,9614-1,Pulley Set,1995,1,77,Technic,12
4,9630-1,Simple Mechanisms Set,1997,1,218,Technic,1
5,9916-1,LEGO Electrical Capacitor,1999,1,1,Technic,2
6,10000-1,Guarded Inn,2001,199,256,Lion Knights,1
7,6016-1,Knights' Arsenal,1987,199,37,Lion Knights,1
8,6041-1,Armor Shop,1986,199,114,Lion Knights,1
9,1794-1,Dragon Master Chariot,1994,186,36,Castle,1


#### Con esto sería suficiente, ya relacionamos las 3 tablas. Ahora buscamos: Star Wars

In [50]:
star_wars_quantity = themes_quantity.query("name_theme == 'Star Wars'")

star_wars_quantity

,set_num,name_set,year,theme_id,num_parts,name_theme,quantity
755,30381-1,Imperial TIE Fighter,2018,158,42,Star Wars,1
756,3219-1,TIE Fighter - Mini,2002,158,12,Star Wars,1
757,3340-1,Star Wars #1 - Sith Minifig Pack,2000,158,30,Star Wars,1
758,3341-1,Star Wars #2 - Luke/Han/Boba Minifig Pack,2000,158,25,Star Wars,1
759,3342-1,Star Wars #3 - Troopers/Chewie Minifig Pack,2000,158,25,Star Wars,1
...,...,...,...,...,...,...,...
1471,8007-1,C-3PO,2001,18,339,Star Wars,1
1472,8008-1,Stormtrooper,2001,18,360,Star Wars,1
1473,8009-1,R2-D2,2002,18,239,Star Wars,1
1474,8010-1,Darth Vader,2002,18,388,Star Wars,1


#### Calculamos su cantidad:


In [51]:
star_wars_quantity['quantity']. sum()

196

#### Extra: Ordenamos de forma ascendente las repeticiones que tienen name_set:

In [52]:
star_wars_quantity.sort_values(by = 'quantity', ascending = False)

,set_num,name_set,year,theme_id,num_parts,name_theme,quantity
871,7913-1,Clone Trooper Battle Pack,2011,158,85,Star Wars,4
872,7914-1,Mandalorian Battle Pack,2011,158,68,Star Wars,4
885,9488-1,Elite Clone Trooper & Commando Droid Battle Pack,2012,158,105,Star Wars,3
810,7200-1,Final Duel I,2002,158,32,Star Wars,3
789,7111-1,Droid Fighter,1999,158,62,Star Wars,3
...,...,...,...,...,...,...,...
815,7255-1,General Grievous Chase,2005,158,112,Star Wars,1
809,7190-1,Millennium Falcon,2000,158,671,Star Wars,1
808,7171-1,Mos Espa Podrace,1999,158,913,Star Wars,1
807,7163-1,Republic Gunship,2002,158,697,Star Wars,1
